# Import libraries

In [1]:
# ignore legacy warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# import libraries
import pandas as pd
import lightgbm as lgb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import csv
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import threading
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Data Description and Cleansing

In [3]:
# Get the data
df = pd.read_csv("census-income-training.csv")

In [4]:
# check what is in the file
df.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,year,income_morethan_50K,Id
0,49,Private,43,2,Doctorate degree(PhD EdD),0,Not in universe,Married-civilian spouse present,Education,Executive admin and managerial,...,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,1,1
1,49,Private,45,12,Bachelors degree(BA AB BS),0,Not in universe,Married-civilian spouse present,Other professional services,Professional specialty,...,United-States,United-States,Native- Born in the United States,0,No,1,30,95,0,2
2,49,Private,12,26,High school graduate,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Adm support including clerical,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0,3
3,38,Private,3,34,High school graduate,0,Not in universe,Married-civilian spouse present,Mining,Precision production craft & repair,...,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,0,5
4,46,Not in universe,0,0,Masters degree(MA MS MEng MEd MSW MBA),0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,...,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,0,6


Descriptions of Data (From original dataset: https://archive.ics.uci.edu/ml/datasets/Census-Income+%28KDD%29)

age AAGE
class of worker ACLSWKR
industry code ADTIND
occupation code ADTOCC
adjusted gross income AGI
education AHGA
wage per hour AHRSPAY
enrolled in edu inst last wk AHSCOL
marital status AMARITL
major industry code AMJIND
major occupation code AMJOCC
mace ARACE
hispanic Origin AREORGN
sex ASEX
member of a labor union AUNMEM
reason for unemployment AUNTYPE
full or part time employment stat AWKSTAT
capital gains CAPGAIN
capital losses CAPLOSS
divdends from stocks DIVVAL
federal income tax liability FEDTAX
tax filer status FILESTAT
region of previous residence GRINREG
state of previous residence GRINST
detailed household and family stat HHDFMX
detailed household summary in household HHDREL
instance weight MARSUPWT
migration code-change in msa MIGMTR1
migration code-change in reg MIGMTR3
migration code-move within reg MIGMTR4
live in this house 1 year ago MIGSAME
migration prev res in sunbelt MIGSUN
num persons worked for employer NOEMP
family members under 18 PARENT
total person earnings PEARNVAL
country of birth father PEFNTVTY
country of birth mother PEMNTVTY
country of birth self PENATVTY
citizenship PRCITSHP
total person income PTOTVAL
own business or self employed SEOTR
taxable income amount TAXINC
fill inc questionnaire for veteran's admin VETQVA
veterans benefits VETYN
weeks worked in year WKSWORK

In [5]:
print(df.shape)

(142863, 42)


In [6]:
# put Id to the front
cols = df.columns.tolist()
cols = cols[-1:] + cols [:-1]
train = df[cols]
train.head()

,Id,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,year,income_morethan_50K
0,1,49,Private,43,2,Doctorate degree(PhD EdD),0,Not in universe,Married-civilian spouse present,Education,...,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,1
1,2,49,Private,45,12,Bachelors degree(BA AB BS),0,Not in universe,Married-civilian spouse present,Other professional services,...,United-States,United-States,United-States,Native- Born in the United States,0,No,1,30,95,0
2,3,49,Private,12,26,High school graduate,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,...,Canada,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0
3,5,38,Private,3,34,High school graduate,0,Not in universe,Married-civilian spouse present,Mining,...,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,0
4,6,46,Not in universe,0,0,Masters degree(MA MS MEng MEd MSW MBA),0,Not in universe,Married-civilian spouse present,Not in universe or children,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,0


In [7]:
# list all non numeric objects
object_list = []
for col in df.columns:
    if df.dtypes[col] == "object":
        object_list.append(col)
for i in object_list:
    print(f"{i}: {df[i].unique()}")

ACLSWKR: ['Private' 'Not in universe' 'Self-employed-incorporated'
 'Self-employed-not incorporated' 'Local government' 'Never worked'
 'State government' 'Federal government' 'Without pay']
AHGA: ['Doctorate degree(PhD EdD)' 'Bachelors degree(BA AB BS)'
 'High school graduate' 'Masters degree(MA MS MEng MEd MSW MBA)'
 'Some college but no degree' '7th and 8th grade'
 'Associates degree-academic program'
 'Associates degree-occup /vocational' '11th grade' '10th grade'
 '5th or 6th grade' '9th grade' '12th grade no diploma'
 '1st 2nd 3rd or 4th grade' 'Prof school degree (MD DDS DVM LLB JD)'
 'Less than 1st grade']
AHSCOL: ['Not in universe' 'College or university' 'High school']
AMARITL: ['Married-civilian spouse present' 'Divorced' 'Never married' 'Widowed'
 'Married-spouse absent' 'Separated' 'Married-A F spouse present']
AMJIND: ['Education' 'Other professional services' 'Manufacturing-durable goods'
 'Mining' 'Not in universe or children' 'Wholesale trade' 'Retail trade'
 'Entertai

In [8]:
# Descriptive stat for non object columns
df.describe()

,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,HHDREL,NOEMP,SEOTR,VETYN,WKSWORK,year,income_morethan_50K,Id
count,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000
mean,44.701966,20.744048,15.183210,74.378460,603.666184,49.138608,273.175266,1762.973738,2.645976,0.237990,1.986344,31.706971,94.502243,0.085900,107169.572612
std,17.698140,18.150140,14.862149,315.208437,5501.253484,310.486133,2249.896073,1002.945454,2.399373,0.632973,0.116060,23.464480,0.499997,0.280218,61936.100848
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.670000,0.000000,0.000000,1.000000,0.000000,94.000000,0.000000,1.000000
25%,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1078.720000,0.000000,0.000000,2.000000,0.000000,94.000000,0.000000,53572.000000
50%,42.000000,27.000000,12.000000,0.000000,0.000000,0.000000,0.000000,1637.090000,2.000000,0.000000,2.000000,50.000000,95.000000,0.000000,107017.000000
75%,57.000000,37.000000,29.000000,0.000000,0.000000,0.000000,0.000000,2209.065000,5.000000,0.000000,2.000000,52.000000,95.000000,0.000000,160801.000000
max,90.000000,51.000000,46.000000,9999.000000,99999.000000,4608.000000,99999.000000,16349.200000,6.000000,2.000000,2.000000,52.000000,95.000000,1.000000,214602.000000


Abnormal responses on object type columns:

AREORGN (hispanic Origin) = "Do not know"
GRINREG (region of previous residence) = "?"
MIGMTR1 (migration code-change in msa) = "?"
MIGMTR3 (migration code-change in reg) = "?"
MIGSAME (live in this house 1 year ago) = "?"
PARENT (family members under 18) = Only one variable "Not in universe"
PEFNTVTY (country of birth father) = "?"
PEMNTVTY (country of birth mother) = "?"
PENATVTY (country of birth self) = "?"

Dropping those variable can reduce the noise. If there are small number of observation containing those observation, then we'll delete those observations. If not, we'll delete the columns since the feature can't give much information. Also we'll be deleting the Parent feature since it has only 1 variable. But before that we need to check if the test_df has the same problem.

In [51]:
test_df = pd.read_csv("census-income-test.csv")
object_list = []
for col in test_df.columns:
    if test_df.dtypes[col] == "object":
        object_list.append(col)
for i in object_list:
    print(f"{i}: {df[i].unique()}")

ACLSWKR: ['Private' 'Not in universe' 'Self-employed-incorporated'
 'Self-employed-not incorporated' 'Local government' 'Never worked'
 'State government' 'Federal government' 'Without pay']
AHGA: ['Doctorate degree(PhD EdD)' 'Bachelors degree(BA AB BS)'
 'High school graduate' 'Masters degree(MA MS MEng MEd MSW MBA)'
 'Some college but no degree' '7th and 8th grade'
 'Associates degree-academic program'
 'Associates degree-occup /vocational' '11th grade' '10th grade'
 '5th or 6th grade' '9th grade' '12th grade no diploma'
 '1st 2nd 3rd or 4th grade' 'Prof school degree (MD DDS DVM LLB JD)'
 'Less than 1st grade']
AHSCOL: ['Not in universe' 'College or university' 'High school']
AMARITL: ['Married-civilian spouse present' 'Divorced' 'Never married' 'Widowed'
 'Married-spouse absent' 'Separated' 'Married-A F spouse present']
AMJIND: ['Education' 'Other professional services' 'Manufacturing-durable goods'
 'Mining' 'Not in universe or children' 'Wholesale trade' 'Retail trade'
 'Entertai

KeyError: 'MIGMTR1'

Since the test dataset has the same issues, dropping the observations are not an ideal option. Therefore we'll check the proportion of the abnormal responses and decide whether to drop the feature or not.

In [10]:
print(f"Proportion of 'Do not know' observation in AREORGN: {df.loc[df.AREORGN == 'Do not know', 'AREORGN'].count()/142863}")
print(f"Proportion of '?' observation in GRINREG: {df.loc[df.GRINREG == '?', 'GRINREG'].count()/142863}")
print(f"Proportion of '?' observation in MIGMTR1: {df.loc[df.MIGMTR1 == '?', 'MIGMTR1'].count()/142863}")
print(f"Proportion of '?' observation in MIGMTR3: {df.loc[df.MIGMTR3 == '?', 'MIGMTR3'].count()/142863}")
print(f"Proportion of '?' observation in MIGSAME: {df.loc[df.MIGSAME == '?', 'MIGSAME'].count()/142863}")
print(f"Proportion of '?' observation in PEFNTVTY: {df.loc[df.PEFNTVTY == '?', 'PEFNTVTY'].count()/142863}")
print(f"Proportion of '?' observation in PEMNTVTY: {df.loc[df.PEMNTVTY == '?', 'PEMNTVTY'].count()/142863}")
print(f"Proportion of '?' observation in PENATVTY: {df.loc[df.PENATVTY == '?', 'PENATVTY'].count()/142863}")

Proportion of 'Do not know' observation in AREORGN: 0.0016589319837886647
Proportion of '?' observation in GRINREG: 0.0036678496181656553
Proportion of '?' observation in MIGMTR1: 0.5022434080202711
Proportion of '?' observation in MIGMTR3: 0.5022434080202711
Proportion of '?' observation in MIGSAME: 0.5022434080202711
Proportion of '?' observation in PEFNTVTY: 0.03910039688372777
Proportion of '?' observation in PEMNTVTY: 0.03520155673617382
Proportion of '?' observation in PENATVTY: 0.020950141044217187


MIGMTR1, MIGMTR3, MIGSAME has very high proportion of the abnormal responses (over 0.5). Therefore we'll drop those features with PARENT and ID features.

In [11]:
df = df.drop(['MIGMTR1', 'MIGMTR3', 'MIGSAME', 'PARENT', 'Id'], axis=1)
df.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,year,income_morethan_50K
0,49,Private,43,2,Doctorate degree(PhD EdD),0,Not in universe,Married-civilian spouse present,Education,Executive admin and managerial,...,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,1
1,49,Private,45,12,Bachelors degree(BA AB BS),0,Not in universe,Married-civilian spouse present,Other professional services,Professional specialty,...,United-States,United-States,United-States,Native- Born in the United States,0,No,1,30,95,0
2,49,Private,12,26,High school graduate,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Adm support including clerical,...,Canada,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,0
3,38,Private,3,34,High school graduate,0,Not in universe,Married-civilian spouse present,Mining,Precision production craft & repair,...,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94,0
4,46,Not in universe,0,0,Masters degree(MA MS MEng MEd MSW MBA),0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,...,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,0


In [12]:
X, y = df.iloc[:,:-1] , df.iloc[:,-1]
X.head()

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,NOEMP,PEFNTVTY,PEMNTVTY,PENATVTY,PRCITSHP,SEOTR,VETQVA,VETYN,WKSWORK,year
0,49,Private,43,2,Doctorate degree(PhD EdD),0,Not in universe,Married-civilian spouse present,Education,Executive admin and managerial,...,3,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94
1,49,Private,45,12,Bachelors degree(BA AB BS),0,Not in universe,Married-civilian spouse present,Other professional services,Professional specialty,...,6,United-States,United-States,United-States,Native- Born in the United States,0,No,1,30,95
2,49,Private,12,26,High school graduate,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Adm support including clerical,...,6,Canada,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94
3,38,Private,3,34,High school graduate,0,Not in universe,Married-civilian spouse present,Mining,Precision production craft & repair,...,6,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94
4,46,Not in universe,0,0,Masters degree(MA MS MEng MEd MSW MBA),0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,...,0,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95


In [13]:
y.head()

0    1
1    0
2    0
3    0
4    0
Name: income_morethan_50K, dtype: int64

In [14]:
X.describe()

,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,HHDREL,NOEMP,SEOTR,VETYN,WKSWORK,year
count,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000,142863.000000
mean,44.701966,20.744048,15.183210,74.378460,603.666184,49.138608,273.175266,1762.973738,2.645976,0.237990,1.986344,31.706971,94.502243
std,17.698140,18.150140,14.862149,315.208437,5501.253484,310.486133,2249.896073,1002.945454,2.399373,0.632973,0.116060,23.464480,0.499997
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.670000,0.000000,0.000000,1.000000,0.000000,94.000000
25%,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1078.720000,0.000000,0.000000,2.000000,0.000000,94.000000
50%,42.000000,27.000000,12.000000,0.000000,0.000000,0.000000,0.000000,1637.090000,2.000000,0.000000,2.000000,50.000000,95.000000
75%,57.000000,37.000000,29.000000,0.000000,0.000000,0.000000,0.000000,2209.065000,5.000000,0.000000,2.000000,52.000000,95.000000
max,90.000000,51.000000,46.000000,9999.000000,99999.000000,4608.000000,99999.000000,16349.200000,6.000000,2.000000,2.000000,52.000000,95.000000


In [15]:
for col in X.columns:
    if X.dtypes[col] != "object":
        print(col)

AAGE
ADTIND
ADTOCC
AHRSPAY
CAPGAIN
CAPLOSS
DIVVAL
HHDREL
NOEMP
SEOTR
VETYN
WKSWORK
year


Since we are going to use XGBboosting, we need to encode categorical variables into numerical variable. The two vays to encode categorical variables are 1.One-hot Encoding, and 2.Label-encoding. Each encoding methods used in different situation, one-hot encoding when the number of unique variable of the columns are few and label encoding when the unique variable are ordial variable. But for convenience, we'll do one hot encoding to all of the object variable and the order of ordial variable is not defined.

Features with dtype == object
ACLSWKR - One-hot v
AHGA - One-hot v
AHSCOL - One-hot
AMARITL - One-hot v
AMJIND - One-hot 
AMJOCC - One-hot 
ARACE - One-hot v
AREORGN - One-hot
ASEX - One-hot v
AUNMEM - One-hot v
AUNTYPE - One-hot
AWKSTAT - One-hot v
FEDTAX - One-hot
FILESTAT - One-hot
GRINREG - One-hot
GRINST - One-hot
HHDFMX - One-hot
MIGMTR4 - One-hot
PEFNTVTY - One-hot v
PEMNTVTY - One-hot
PENATVTY - One-hot
PRCITSHP - One-hot v
VETQVA - One-hot

Feature with dtype != object:
AAGE v
ADTIND v
ADTOCC v
AHRSPAY v
CAPGAIN v
CAPLOSS v 
DIVVAL v
HHDREL
NOEMP v
SEOTR 
VETYN v
WKSWORK v

Since there are many features in this dataset, we'll go through one model with all the features and the other one with key features (with v mark)

In [16]:
X_key = X.loc[:,['ACLSWKR','AMARITL','ARACE','ASEX','AUNMEM','AWKSTAT',
                 'PEFNTVTY','PRCITSHP','AAGE','ADTIND','ADTOCC','AHRSPAY',
                 'CAPGAIN','CAPLOSS','DIVVAL','NOEMP','VETYN','WKSWORK']]
X_key.head()

,ACLSWKR,AMARITL,ARACE,ASEX,AUNMEM,AWKSTAT,PEFNTVTY,PRCITSHP,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,NOEMP,VETYN,WKSWORK
0,Private,Married-civilian spouse present,White,Male,Not in universe,Children or Armed Forces,United-States,Native- Born in the United States,49,43,2,0,0,0,0,3,2,52
1,Private,Married-civilian spouse present,Black,Male,Not in universe,Full-time schedules,United-States,Native- Born in the United States,49,45,12,0,0,0,0,6,1,30
2,Private,Married-civilian spouse present,White,Female,No,Children or Armed Forces,Canada,Native- Born in the United States,49,12,26,0,0,0,50,6,2,52
3,Private,Married-civilian spouse present,White,Male,Not in universe,Children or Armed Forces,United-States,Native- Born in the United States,38,3,34,0,0,0,0,6,2,52
4,Not in universe,Married-civilian spouse present,Black,Female,Not in universe,Not in labor force,United-States,Native- Born in the United States,46,0,0,0,0,0,0,0,2,0


We'll now go through encoding part of two dataframe X, and X_key.

In [17]:
object_list = []
for col in X.columns:
    if X.dtypes[col] == "object":
        object_list.append(col)

for i in object_list:
    dummy = pd.get_dummies(X[i], prefix= f"{i}_")
    X = pd.merge(left=X, right=dummy,
                 left_index= True, right_index= True)

X = X.drop(object_list, axis=1)
X.head()


,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,HHDREL,NOEMP,SEOTR,...,PENATVTY__Vietnam,PENATVTY__Yugoslavia,PRCITSHP__Foreign born- Not a citizen of U S,PRCITSHP__Foreign born- U S citizen by naturalization,PRCITSHP__Native- Born abroad of American Parent(s),PRCITSHP__Native- Born in Puerto Rico or U S Outlying,PRCITSHP__Native- Born in the United States,VETQVA__No,VETQVA__Not in universe,VETQVA__Yes
0,49,43,2,0,0,0,0,1837.06,3,2,...,0,0,0,0,0,0,1,0,1,0
1,49,45,12,0,0,0,0,372.48,6,0,...,0,0,0,0,0,0,1,1,0,0
2,49,12,26,0,0,0,50,519.79,6,0,...,0,0,0,0,0,0,1,0,1,0
3,38,3,34,0,0,0,0,327.99,6,2,...,0,0,0,0,0,0,1,0,1,0
4,46,0,0,0,0,0,0,1893.23,0,0,...,0,0,0,0,0,0,1,0,1,0


In [18]:
object_list = []
for col in X_key.columns:
    if X_key.dtypes[col] == "object":
        object_list.append(col)

for i in object_list:
    dummy = pd.get_dummies(X_key[i], prefix= f"{i}_")
    X_key = pd.merge(left=X_key, right=dummy,
                 left_index= True, right_index= True)

X_key = X_key.drop(object_list, axis=1)
X_key.head()

,AAGE,ADTIND,ADTOCC,AHRSPAY,CAPGAIN,CAPLOSS,DIVVAL,NOEMP,VETYN,WKSWORK,...,PEFNTVTY__Thailand,PEFNTVTY__Trinadad&Tobago,PEFNTVTY__United-States,PEFNTVTY__Vietnam,PEFNTVTY__Yugoslavia,PRCITSHP__Foreign born- Not a citizen of U S,PRCITSHP__Foreign born- U S citizen by naturalization,PRCITSHP__Native- Born abroad of American Parent(s),PRCITSHP__Native- Born in Puerto Rico or U S Outlying,PRCITSHP__Native- Born in the United States
0,49,43,2,0,0,0,0,3,2,52,...,0,0,1,0,0,0,0,0,0,1
1,49,45,12,0,0,0,0,6,1,30,...,0,0,1,0,0,0,0,0,0,1
2,49,12,26,0,0,0,50,6,2,52,...,0,0,0,0,0,0,0,0,0,1
3,38,3,34,0,0,0,0,6,2,52,...,0,0,1,0,0,0,0,0,0,1
4,46,0,0,0,0,0,0,0,2,0,...,0,0,1,0,0,0,0,0,0,1


In [19]:
# split the dataset by ourselves
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 3400)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 3400)

In [20]:
y_test.head()

55550     0
12431     0
122464    0
57508     0
71023     0
Name: income_morethan_50K, dtype: int64

Define the function to evaluate models

In [21]:
def classifier_eval(test, pred):
    print(f"Confusion Matrix: {confusion_matrix(test, pred)}")
    print(f"Accuracy: {accuracy_score(test, pred)}")
    print(f"Precision: {precision_score(test, pred)}")
    print(f"Recall: {recall_score(test, pred)}")
    print(f"F1: {f1_score(test, pred)}")
    print(f"ROC_AUC_SCORE: {roc_auc_score(test, pred)}")

Preprocess test_df as well

In [52]:
preprocessed_test_df = test_df.drop(['MIGMTR1', 'MIGMTR3', 'MIGSAME', 'PARENT', 'Id'], axis=1)

object_list = []
for col in preprocessed_test_df.columns:
    if preprocessed_test_df.dtypes[col] == "object":
        object_list.append(col)

for i in object_list:
    dummy = pd.get_dummies(preprocessed_test_df[i], prefix= f"{i}_")
    preprocessed_test_df = pd.merge(left=preprocessed_test_df, right=dummy,
                 left_index= True, right_index= True)

preprocessed_test_df = preprocessed_test_df.drop(object_list, axis=1)

# XGB Classifier

In [ ]:
# Create the parameter grid
param_grid = {
    'n_estimators': [300, 400, 500],
    'learning_rate': [0.1],
    'max_depth': [1,2,3],
}

# Create a based model
rf = xgb.XGBClassifier(use_label_encoder=False)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1, cv=2)

# search what is the best hyperparameter value
grid_search.fit(X_train, y_train, early_stopping_rounds = 100,
              eval_metric="logloss", 
              eval_set=[(X_val, y_val)], verbose=False)

print(grid_search.best_params_)

Now we choose learning_rate = 0.1, max_depth = 3 and estimators = 500 as our parameters

In [ ]:
xgb_model= xgb.XGBClassifier(n_estimators = 500, learning_rate = 0.1, max_depth = 3, use_label_encoder=False)

In [ ]:
xgb_model.fit(X_train, y_train, 
              early_stopping_rounds = 100,
              eval_metric="logloss", 
              eval_set=[(X_val, y_val)])

In [ ]:
y_pred = xgb_model.predict(X_test)

In [ ]:
classifier_eval(y_test, y_pred)

# Result of XGB classifier:

Confusion Matrix: [[25671   391]
 [ 1373  1138]]
Accuracy: 0.9382633955132468
Precision: 0.7442773054283846
Recall: 0.4532058940661091
F1: 0.5633663366336633
ROC_AUC_SCORE: 0.7191016040816312

# Logistic Regression

In [ ]:
# Create the parameter grid
param_grid = {
    'penalty': ['l2', 'elasticnet'],
    'solver' : ['sag', 'saga'],
    'max_iter' : [100000]
}

# Create a based model
rf = LogisticRegression()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1, cv=2)

# search what is the best hyperparameter value
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

We use the best performing parameters: penalty = l2 and solver = sag.

In [ ]:
lregressionmodel = LogisticRegression(solver='sag',penalty='l2', max_iter = 10000)
lregressionmodel.fit(X_train, y_train)

In [ ]:
y_pred = lregressionmodel.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Logistic Regression:


Confusion Matrix: [[25988    74]
 [ 2110   401]]
Accuracy: 0.9235642039687817
Precision: 0.8442105263157895
Recall: 0.15969733174034248
F1: 0.2685867381111855
ROC_AUC_SCORE: 0.5784289743652983

# Random Forest

In [ ]:
random_forest = RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 8, n_estimators= 700)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Random Forest
Confusion Matrix: [[26062     0]
 [ 2511     0]]
Accuracy: 0.9121198334091625
Precision: 0.0
Recall: 0.0
F1: 0.0
ROC_AUC_SCORE: 0.5

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of KNN


Confusion Matrix: [[25228   834]
 [ 1715   796]]
Accuracy: 0.9107899065551395
Precision: 0.4883435582822086
Recall: 0.31700517722023097
F1: 0.38444820091765275
ROC_AUC_SCORE: 0.6425022816497901

# Gaussian NB

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
y_pred = gaussian.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Gaussian NB

Confusion Matrix: [[18016  8046]
 [  246  2265]]
Accuracy: 0.7097959612221327
Precision: 0.21966831539132964
Recall: 0.9020310633213859
F1: 0.35329901731399155
ROC_AUC_SCORE: 0.7966528580362591

# Support Vector Machine

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Support Vector Machine

Confusion Matrix: [[25567   495]
 [ 2081   430]]
Accuracy: 0.909844958527281
Precision: 0.4648648648648649
Recall: 0.17124651533253685
F1: 0.25029103608847497
ROC_AUC_SCORE: 0.5761266725998883

# Decision Tree

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Decision Tree

Confusion Matrix: [[24656  1406]
 [ 1341  1170]]
Accuracy: 0.9038602876841774
Precision: 0.45419254658385094
Recall: 0.4659498207885305
F1: 0.4599960684096717
ROC_AUC_SCORE: 0.7060007718016783

# AdaBoost

In [ ]:
# Create the parameter grid based on the results of random search
param_grid = {
    'n_estimators': [500, 600, 700],
}
# Create a based model
rf = AdaBoostClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

Test for what is the best performing estimator, the best parameter for n_estimators is 700.

In [ ]:
ada_boost = AdaBoostClassifier(n_estimators=700, random_state=0)
ada_boost.fit(X_train, y_train)
y_pred = ada_boost.predict(X_test)
classifier_eval(y_test, y_pred)

# Result of Ada Boost

Confusion Matrix: [[25651   411]
 [ 1419  1092]]
Accuracy: 0.9359535225562594
Precision: 0.7265469061876247
Recall: 0.4348864994026284
F1: 0.5440956651718983
ROC_AUC_SCORE: 0.7095582063431684

# Gradient Boosting

In [55]:
# Create the parameter grid based on the results of random search
param_grid = {
    
    'n_estimators': [500, 600, 700],
    'max_depth': [4, 5, 6],
    'learning_rate': [0.1],
}
# Create a based model
rf = GradientBoostingClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=2, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=600; total time= 3.0min
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=600; total time= 3.0min
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 500}
[CV] END ...learning_rate=0.1, max_depth=4, n_estimators=600; total time= 2.4min
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=500; total time= 2.6min
[CV] END ...learning_rate=0.1, max_depth=4, n_estimators=600; total time= 2.4min
[CV] END ...learning_rate=0.1, max_depth=6, n_estimators=500; total time= 2.6min
[CV] END ...learning_rate=0.1, max_depth=4, n_estimators=500; total time= 2.0min
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=700; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=4, n_estimators=500; total time= 2.0min
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=700; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=5, n_estimators=500; total ti

Test for what is the best performing estimator, the best parameter for n_estimators is n_estimator = 500, max_depth = 4 and learning rate = 0.1.

In [22]:
gradient_boosting = GradientBoostingClassifier(n_estimators=500, max_depth=4, learning_rate=0.1)
gradient_boosting.fit(X_train, y_train)
y_pred = gradient_boosting.predict(X_test)

In [25]:
classifier_eval(y_test, y_pred)

Confusion Matrix: [[25606   456]
 [ 1294  1217]]
Accuracy: 0.9387533685647289
Precision: 0.7274357441721458
Recall: 0.4846674631620868
F1: 0.5817399617590823
ROC_AUC_SCORE: 0.7335853623077719


# Result of Gradient Boosting

Confusion Matrix: [[25606   456]
 [ 1294  1217]]
Accuracy: 0.9387533685647289
Precision: 0.7274357441721458
Recall: 0.4846674631620868
F1: 0.5817399617590823
ROC_AUC_SCORE: 0.7335853623077719

# Given good result, submit to kaggle. Kaggle result: 0.93954

In [ ]:
predicted_result = gradient_boosting.predict(preprocessed_test_df)
my_submission = pd.DataFrame({'Id': test_df['Id'], 'income_morethan_50K': predicted_result})
my_submission = my_submission.sort_values(by=["Id"])
print(my_submission.head())
my_submission.to_csv("submission_gradient_boosting.csv", index=False)

# Light GBM

In [43]:
# Create the parameter grid
param_grid = {
    # other tested: 0.2, 0.3, 0.5
    'learning_rate': [0.08, 0.09, 0.1],
    # other tested: -10000, -5000, -3000, -2000, -1800, -1500, -1000, -800, -400, -300, -200, -100, -50, -30, -20, -5
    'max_depth': [-1000000, -100000, -10000, -5000, -3000, -2000, ],
}

# Create a based model
rf = lgb.LGBMClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, n_jobs=-1, cv=2)

# search what is the best hyperparameter value
grid_search.fit(X_train, y_train, early_stopping_rounds = 100,
              eval_metric="logloss", 
              eval_set=[(X_val, y_val)], verbose=False)

print(grid_search.best_params_)

/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'earl

/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'earl

/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'earl

{'learning_rate': 0.1, 'max_depth': -1000000}


The best parameter is learning_rate = 0.1 and max_depth = -1000000

In [44]:
# create model
model = lgb.LGBMClassifier(learning_rate=0.1,max_depth=-1000000,random_state=42)
model.fit(X_train,y_train,eval_set=[(X_val, y_val),(X_train,y_train)],
          verbose=20,eval_metric='logloss')
y_pred = model.predict(X_test)

classifier_eval(y_test, y_pred)

/Users/brian/miniforge3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[20]	training's binary_logloss: 0.168312	valid_0's binary_logloss: 0.175337
[40]	training's binary_logloss: 0.152405	valid_0's binary_logloss: 0.164184
[60]	training's binary_logloss: 0.145192	valid_0's binary_logloss: 0.160763
[80]	training's binary_logloss: 0.140083	valid_0's binary_logloss: 0.159503
[100]	training's binary_logloss: 0.136194	valid_0's binary_logloss: 0.159297
Confusion Matrix: [[25652   410]
 [ 1346  1165]]
Accuracy: 0.9385433801140938
Precision: 0.7396825396825397
Recall: 0.4639585822381521
F1: 0.5702398433675966
ROC_AUC_SCORE: 0.7241134327812663
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] 

# Result of Light GBM

Confusion Matrix: [[25652   410]
 [ 1346  1165]]
Accuracy: 0.9385433801140938
Precision: 0.7396825396825397
Recall: 0.4639585822381521
F1: 0.5702398433675966
ROC_AUC_SCORE: 0.7241134327812663

# Given good result, submit to kaggle. Kaggle result: 0.93896

In [53]:
predicted_result = model.predict(preprocessed_test_df)
my_submission = pd.DataFrame({'Id': test_df['Id'], 'income_morethan_50K': predicted_result})
my_submission = my_submission.sort_values(by=["Id"])
my_submission.to_csv("submission_light_gbm.csv", index=False)

   Id  income_morethan_50K
0   4                    0
1   7                    0
2  16                    0
3  17                    0
4  22                    0
